# Mục tiêu : Đọc và xây dựng chuyển thành input đầu vào

* Fb15k
* Fb15k-237
* WN18
* WN18RR

In [87]:
def read_from_id(filename='../dataset/WN18RR/entity2id.txt'):
    entity2id = {}
    id2entity = {}
    with open(filename) as f:
        for line in f:
            if len(line.strip().split()) > 1:
                tmp = line.strip().split()
                entity2id[tmp[0]] = int(tmp[1])
                id2entity[int(tmp[1])] = tmp[0]
    return entity2id, id2entity

In [89]:
def init_norm_Vector(relinit, entinit, embedding_size):
    lstent = []
    lstrel = []
    with open(relinit) as f:
        for line in f:
            tmp = [float(val) for val in line.strip().split()]
            # if np.linalg.norm(tmp) > 1:
            #     tmp = tmp / np.linalg.norm(tmp)
            lstrel.append(tmp)
    with open(entinit) as f:
        for line in f:
            tmp = [float(val) for val in line.strip().split()]
            # if np.linalg.norm(tmp) > 1:
            #     tmp = tmp / np.linalg.norm(tmp)
            lstent.append(tmp)
    assert embedding_size % len(lstent[0]) == 0
    return np.array(lstent, dtype=np.float32), np.array(lstrel, dtype=np.float32)

In [ ]:
def getID(folder='data/WN18RR/'):
    lstEnts = {}
    lstRels = {}
    with open(folder + 'train.txt') as f:
        for line in f:
            line = line.strip().split()
            if line[0] not in lstEnts:
                lstEnts[line[0]] = len(lstEnts)
            if line[2] not in lstEnts:
                lstEnts[line[2]] = len(lstEnts)
            if line[1] not in lstRels:
                lstRels[line[1]] = len(lstRels)

    with open(folder + 'valid.txt') as f:
        for line in f:
            line = line.strip().split()
            if line[0] not in lstEnts:
                lstEnts[line[0]] = len(lstEnts)
            if line[2] not in lstEnts:
                lstEnts[line[2]] = len(lstEnts)
            if line[1] not in lstRels:
                lstRels[line[1]] = len(lstRels)

    with open(folder + 'test.txt') as f:
        for line in f:
            line = line.strip().split()
            if line[0] not in lstEnts:
                lstEnts[line[0]] = len(lstEnts)
            if line[2] not in lstEnts:
                lstEnts[line[2]] = len(lstEnts)
            if line[1] not in lstRels:
                lstRels[line[1]] = len(lstRels)

    wri = open(folder + 'entity2id.txt', 'w')
    for entity in lstEnts:
        wri.write(entity + '\t' + str(lstEnts[entity]))
        wri.write('\n')
    wri.close()

    wri = open(folder + 'relation2id.txt', 'w')
    for entity in lstRels:
        wri.write(entity + '\t' + str(lstRels[entity]))
        wri.write('\n')
    wri.close()

In [80]:
def parse_line(line):
    line = line.strip().split()
    sub = line[0]
    rel = line[1]
    obj = line[2]
    val = [1]
    if len(line) > 3:
        if line[3] == '-1':
            val = [-1]
    return sub, obj, rel, val

In [90]:
def load_triples_from_txt(filename, words_indexes=None, parse_line=parse_line):
    """
    Take a list of file names and build the corresponding dictionnary of triples
    """
    if words_indexes == None:
        words_indexes = dict()
        entities = set()
        next_ent = 0
    else:
        entities = set(words_indexes)
        next_ent = max(words_indexes.values()) + 1

    data = dict()

    with open(filename) as f:
        lines = f.readlines()

    for _, line in enumerate(lines):
        sub, obj, rel, val = parse_line(line)

        if sub in entities:
            sub_ind = words_indexes[sub]
        else:
            sub_ind = next_ent
            next_ent += 1
            words_indexes[sub] = sub_ind
            entities.add(sub)

        if rel in entities:
            rel_ind = words_indexes[rel]
        else:
            rel_ind = next_ent
            next_ent += 1
            words_indexes[rel] = rel_ind
            entities.add(rel)

        if obj in entities:
            obj_ind = words_indexes[obj]
        else:
            obj_ind = next_ent
            next_ent += 1
            words_indexes[obj] = obj_ind
            entities.add(obj)

        data[(sub_ind, rel_ind, obj_ind)] = val

    indexes_words = {}
    for tmpkey in words_indexes:
        indexes_words[words_indexes[tmpkey]] = tmpkey

    return data, words_indexes, indexes_words

In [92]:
def load_triples_from_txt(filename, words_indexes=None, parse_line=parse_line):
    """
    Take a list of file names and build the corresponding dictionnary of triples
    """
    if words_indexes == None:
        words_indexes = dict()
        entities = set()
        next_ent = 0
    else:
        entities = set(words_indexes)
        next_ent = max(words_indexes.values()) + 1

    data = dict()

    with open(filename) as f:
        lines = f.readlines()

    for _, line in enumerate(lines):
        sub, obj, rel, val = parse_line(line)

        if sub in entities:
            sub_ind = words_indexes[sub]
        else:
            sub_ind = next_ent
            next_ent += 1
            words_indexes[sub] = sub_ind
            entities.add(sub)

        if rel in entities:
            rel_ind = words_indexes[rel]
        else:
            rel_ind = next_ent
            next_ent += 1
            words_indexes[rel] = rel_ind
            entities.add(rel)

        if obj in entities:
            obj_ind = words_indexes[obj]
        else:
            obj_ind = next_ent
            next_ent += 1
            words_indexes[obj] = obj_ind
            entities.add(obj)

        data[(sub_ind, rel_ind, obj_ind)] = val

    indexes_words = {}
    for tmpkey in words_indexes:
        indexes_words[words_indexes[tmpkey]] = tmpkey

    return data, words_indexes, indexes_words

In [94]:
def dic_of_chars(words_indexes):
    lstChars = {}
    for word in words_indexes:
        for char in word:
            if char not in lstChars:
                lstChars[char] = len(lstChars)
    lstChars['unk'] = len(lstChars)
    return lstChars

In [95]:
def convert_to_seq_chars(x_batch, lstChars, indexes_words):
    lst = []
    for [tmpH, tmpR, tmpT] in x_batch:
        wH = [lstChars[tmp] for tmp in indexes_words[tmpH]]
        wR = [lstChars[tmp] for tmp in indexes_words[tmpR]]
        wT = [lstChars[tmp] for tmp in indexes_words[tmpT]]
        lst.append([wH, wR, wT])
    return lst

In [96]:
def _pad_sequences(sequences, pad_tok, max_length):
    sequence_padded, sequence_length = [], []
    for seq in sequences:
        seq = list(seq)
        seq_ = seq[:max_length] + [pad_tok] * max(max_length - len(seq), 0)
        sequence_padded += [seq_]
        sequence_length += [min(len(seq), max_length)]

    return sequence_padded, sequence_length

In [98]:
def pad_sequences(sequences, pad_tok):
    sequence_padded, sequence_length = [], []
    max_length_word = max([max(map(lambda x: len(x), seq))
                           for seq in sequences])
    for seq in sequences:
        # all words are same length now
        sp, sl = _pad_sequences(seq, pad_tok, max_length_word)
        sequence_padded += [sp]
        sequence_length += [sl]

    max_length_sentence = max(map(lambda x: len(x), sequences))
    sequence_padded, _ = _pad_sequences(sequence_padded, [pad_tok] * max_length_word, max_length_sentence)
    sequence_length, _ = _pad_sequences(sequence_length, 0, max_length_sentence)

    return np.array(sequence_padded).astype(np.int32), np.array(sequence_length).astype(np.int32)

In [99]:
def build_data(name='WN18', path='../data'):
    folder = path + '/' + name + '/'

    train_triples, words_indexes, _ = load_triples_from_txt(folder + 'train.txt', parse_line=parse_line)

    valid_triples, words_indexes, _ = load_triples_from_txt(folder + 'valid.txt',
                                                            words_indexes=words_indexes, parse_line=parse_line)

    test_triples, words_indexes, indexes_words = load_triples_from_txt(folder + 'test.txt',
                                                                       words_indexes=words_indexes,
                                                                       parse_line=parse_line)

    entity2id, id2entity = read_from_id(folder + '/entity2id.txt')
    relation2id, id2relation = read_from_id(folder + '/relation2id.txt')
    left_entity = {}
    right_entity = {}

    with open(folder + 'train.txt') as f:
        lines = f.readlines()
    for _, line in enumerate(lines):
        head, tail, rel, val = parse_line(line)
        # count the number of occurrences for each (heal, rel)
        if relation2id[rel] not in left_entity:
            left_entity[relation2id[rel]] = {}
        if entity2id[head] not in left_entity[relation2id[rel]]:
            left_entity[relation2id[rel]][entity2id[head]] = 0
        left_entity[relation2id[rel]][entity2id[head]] += 1
        # count the number of occurrences for each (rel, tail)
        if relation2id[rel] not in right_entity:
            right_entity[relation2id[rel]] = {}
        if entity2id[tail] not in right_entity[relation2id[rel]]:
            right_entity[relation2id[rel]][entity2id[tail]] = 0
        right_entity[relation2id[rel]][entity2id[tail]] += 1

    left_avg = {}
    for i in range(len(relation2id)):
        left_avg[i] = sum(left_entity[i].values()) * 1.0 / len(left_entity[i])

    right_avg = {}
    for i in range(len(relation2id)):
        right_avg[i] = sum(right_entity[i].values()) * 1.0 / len(right_entity[i])

    headTailSelector = {}
    for i in range(len(relation2id)):
        headTailSelector[i] = 1000 * right_avg[i] / (right_avg[i] + left_avg[i])

    return train_triples, valid_triples, test_triples, words_indexes, indexes_words, headTailSelector, entity2id, id2entity, relation2id, id2relation

In [6]:
build_data()

NameError: name 'build_data' is not defined

In [1]:
import numpy as np
a = np.array([1,2,3])

In [11]:
import torch
b = torch.tensor([[1, 2, 3], [4, 5, 6]])
b

tensor([[1, 2, 3],
        [4, 5, 6]])

In [16]:
b.size()

torch.Size([2, 3])

In [17]:
c = b.unsqueeze(-1)
c

tensor([[[1],
         [2],
         [3]],

        [[4],
         [5],
         [6]]])

In [15]:
c.size()

torch.Size([2, 3, 1])

In [18]:
[[434, 6, 0, 20151], [434, 6, 1, 27022], [434, 1, 3, 6756], [434, 1, 1, 9701], [434, 1, 3, 25215], [434, 1, 3, 14514], [434, 1, 1, 17355], [434, 1, 1, 4678], [434, 1, 0, 22969], [434, 0, 0, 8093], [102, 1, 1, 8198], [102, 1, 9, 3513], [102, 1, 9, 6835], [102, 0, 0, 28374], [1015, 1, 1, 9837], [1015, 1, 1, 8193], [1015, 9, 1, 9709], [1015, 9, 1, 24451], [1015, 9, 0, 30263], [1015, 5, 5, 3276], [1015, 5, 0, 24555], [1015, 5, 1, 18987], [1015, 5, 1, 12226], [1015, 0, 0, 1856], [1334, 0, 0, 21207], [1170, 1, 0, 2021], [1170, 0, 1, 5746], [1170, 0, 1, 3407], [1170, 0, 1, 15941], [1170, 0, 3, 17561], [1170, 9, 0, 13432], [1170, 5, 0, 2340], [1170, 5, 1, 432], [551, 1, 1, 1750], [551, 1, 1, 1650], [551, 1, 1, 57], [593, 1, 1, 14724], [593, 1, 1, 15194], [593, 1, 0, 3538], [593, 1, 5, 212], [593, 0, 0, 20441], [878, 1, 10, 15612], [878, 1, 6, 12894], [878, 1, 1, 21993], [878, 1, 1, 7965], [878, 1, 6, 25573], [878, 1, 1, 1353], [878, 1, 0, 267], [878, 1, 1, 1354], [878, 1, 0, 15983], [878, 1, 1, 12156], [878, 1, 1, 4130], [878, 1, 1, 3621], [878, 1, 1, 16831], [878, 1, 0, 6991], [878, 1, 1, 4331], [878, 1, 1, 23169], [878, 0, 1, 5550], [878, 0, 1, 18401], [878, 0, 1, 9048], [878, 0, 1, 6526], [878, 0, 1, 411], [878, 0, 1, 23994], [878, 0, 1, 26005], [878, 1, 0, 5224], [1745, 0, 0, 9550], [1745, 0, 0, 425], [68, 1, 0, 25065], [68, 3, 1, 16639], [68, 3, 1, 2060], [68, 3, 3, 4087], [68, 3, 3, 16968], [68, 3, 3, 2432], [68, 3, 3, 4128], [68, 3, 0, 17329], [68, 3, 1, 31988], [68, 3, 1, 2185], [68, 3, 3, 12153], [392, 3, 1, 8768], [392, 3, 3, 14552], [392, 3, 1, 3825], [392, 3, 10, 5770], [392, 3, 1, 25461], [392, 3, 3, 28623], [392, 3, 1, 15110], [392, 3, 3, 13343], [392, 1, 0, 33642], [392, 1, 1, 10644], [392, 3, 3, 1388], [392, 3, 3, 10720], [392, 3, 3, 14121], [392, 3, 1, 3901], [689, 1, 1, 6041], [689, 1, 4, 608], [689, 1, 0, 11666], [689, 1, 4, 24498], [689, 0, 1, 12211], [689, 0, 1, 838], [689, 0, 3, 25215], [689, 0, 0, 12252...

SyntaxError: invalid syntax (<ipython-input-18-f2d4c0d9b846>, line 1)

In [35]:
import torch
a = torch.tensor([1,2,3])
b = torch.tensor([2,3,4])

class A:
    def funcA(a, b):
        c = torch.tensor([0,0,0])
        c = a + b
        return c

In [27]:
a

tensor([1, 2, 3])

In [28]:
b

tensor([2, 3, 4])

In [29]:
funcA(a, b)

tensor([2, 3, 4])

In [30]:
a

tensor([1, 2, 3])

In [34]:
d = funcA(a,b)
d

tensor([2, 3, 4])